In [11]:
import os, glob, json, spacy, random
import xml.etree.ElementTree as ET

import numpy as np

In [12]:
nlp = spacy.load('pt_core_news_lg')

In [13]:
with open(r'../data/external/CDSegundoHAREM.xml', 'r', encoding='latin9') as arq:
    harem = '\n'.join(arq.readlines())

In [14]:
tree = ET.parse(r'../data/external/CDSegundoHAREM.xml')

In [15]:
def innertext(tag):
    return (tag.text or '') + ''.join(innertext(e) for e in tag) + (tag.tail or '')

In [16]:
def entities(tag):
    return [(e.tag, e.attrib['CATEG'] if 'CATEG' in e.attrib else '', e.text) for e in tag if e.tag == 'EM'] +\
[entities(e) for e in tag]

In [17]:
def flatten_ents(doc):
    ents = []
    for e in entities(doc):
        for j in e:
            if type(j) == list and len(j) > 0:
                for i in list(np.array(j).flatten()):
                    if i:
                        ents.append(i) 
            else:
                if j:
                    ents.append(j)
    return ents

In [18]:
def find_all(a_str, sub):
    start = 0
    while True:
        start = a_str.find(sub, start)
        if start == -1: return
        yield start
        start += len(sub)

In [127]:
class Interval(object):
    def __init__(self, ini, end, category):
        self.ini = ini
        self.end = end
        self.category = category
        
    def __contains__(self, interval):
        return (self.ini >= interval.ini) and (self.end <= interval.end)
    
    def __repr__(self):
        return "(%d, %d, %s)" % (self.ini, self.end, self.category)
            

In [135]:
[*set([1,2,3])-set([1,2])]

[3]

In [136]:
class Harem2Json():
    def __init__(self, root):
        self.docs = [doc for doc in root.iter('DOC')]
        self.final_json = {}
        self.get_docs()
        self.parse_docs()
        
    def get_docs(self):
        self.docs_trt = []
        for doc in self.docs:
            txt = innertext(doc)
            ents = flatten_ents(doc)
            self.docs_trt.append((txt, ents))
    
    def filter_ents(self, ents):
        filtered = []
        intervals = [Interval(*t) for t in ents]
        print("intervals %d" % len(intervals))
        for i,j in combinations(intervals, 2):
            if i in j:
                filtered.append(i)
        end_intervals = [*(set(intervals) - set(filtered))]
        return [(i.ini, i.end, i.category) for i in end_intervals]
    
    def parse_docs(self):
        self.dataset = []
        self.all_entities = []
        print(len(self.docs_trt))
        for n,d in enumerate(self.docs_trt):
            cnt = d[0]
            ents = []
            for x in d[1]:
                try:
                    idx = list(find_all(cnt,x[-1]))
                except:
                    continue
                for i in idx:
                    if len(x) == 3:
                        #ents.append((i,i+len(x[-1]),x[1],x[-1]))
                        ents.append((i,i+len(x[-1]),x[1]))
                        self.all_entities.append((i,i+len(x[-1]),x[1]))
                    else:
                        #ents.append((i,i+len(x[-1]),x[0],x[-1]))
                        ents.append((i,i+len(x[-1]),x[0]))
                        self.all_entities.append((i,i+len(x[-1]),x[0]))
            print(n)
            ents = self.filter_ents(ents)
            self.dataset.append((cnt, {'entities':list(set(ents))}))
        print("terminou")

In [137]:
h = Harem2Json(tree)

<ipython-input-17-dc6552702ea2>:6: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  for i in list(np.array(j).flatten()):


129
0
intervals 507
1
intervals 21
2
intervals 26
3
intervals 26
4
intervals 48
5
intervals 70
6
intervals 28
7
intervals 384
8
intervals 277
9
intervals 131
10
intervals 221
11
intervals 298
12
intervals 281
13
intervals 217
14
intervals 132
15
intervals 47
16
intervals 790
17
intervals 49
18
intervals 117
19
intervals 105
20
intervals 122
21
intervals 142
22
intervals 164
23
intervals 625
24
intervals 509
25
intervals 94
26
intervals 161
27
intervals 72
28
intervals 220
29
intervals 159
30
intervals 662
31
intervals 148
32
intervals 331
33
intervals 158
34
intervals 127
35
intervals 276
36
intervals 86
37
intervals 140
38
intervals 62
39
intervals 44
40
intervals 42
41
intervals 55
42
intervals 60
43
intervals 139
44
intervals 47
45
intervals 114
46
intervals 90
47
intervals 107
48
intervals 95
49
intervals 103
50
intervals 27
51
intervals 39
52
intervals 54
53
intervals 43
54
intervals 99
55
intervals 42
56
intervals 41
57
intervals 91
58
intervals 31
59
intervals 31
60
intervals 10

In [138]:
h.dataset

[('\n    Fatores Demográficos e Econômicos Subjacentes\n    \n      A revolta histórica produz normalmente uma nova forma de pensamento quanto à forma de organização da sociedade. Assim foi com a\n      Reforma Protestante\n      . No seguimento do colapso de instituições monásticas e do escolasticismo\n      nos finais da Idade Média\n      na\n      Europa\n      , acentuado pela "\n      Cativeiro Babilónica da igreja\n      "\n      no papado de Avignon\n      , o\n      Grande Cisma\n      e o fracasso da conciliação, assistimos\n      no século XVI\n      ao fermentar de um enorme debate sobre a reforma da religião e dos posteriores valores religiosos fundamentais. Este debate passou completamente ao lado de\n      Portugal\n      , demasiado distante do foco onde surgiram estes pensamentos. A imprensa, inventada na\n      Alemanha\n      por\n      John Gutenberg\n      , foi importante na divulgação destas ideias. As  95 Teses de Martinho Lutero | \n      95 Teses \n      de\n 

In [129]:
i1, i2 = Interval(1917, 1929, 'ACONTECIMENTO'), Interval(1917, 1919, 'ACONTECIMENTO')

In [130]:
i1.end <= i2.end

False

In [131]:
i1 in i2

True

In [ ]:
h.dataset

In [86]:
import re

In [98]:
harem

'<?xml version="1.0" encoding="ISO-8859-1"?>\n\n<!DOCTYPE colHAREM>\n\n<colHAREM versao="Segundo_dourada_11Nov2008">\n\n  <DOC DOCID="H2-dftre765">\n\n    <P>Fatores Demográficos e Econômicos Subjacentes</P>\n\n    <P>\n\n      A revolta histórica produz normalmente uma nova forma de pensamento quanto à forma de organização da sociedade. Assim foi com a\n\n      <EM ID="H2-dftre765-1" CATEG="ABSTRACCAO|ACONTECIMENTO" TIPO="IDEIA|EFEMERIDE">Reforma Protestante</EM>\n\n      . No seguimento do colapso de instituições monásticas e do escolasticismo\n\n      nos finais da <EM ID="H2-dftre765-102" CATEG="OUTRO" COMENT="DUVIDA_DIRECTIVASTEMPO">Idade Média</EM>\n\n      na\n\n      <EM ID="H2-dftre765-37" CATEG="LOCAL" TIPO="HUMANO" SUBTIPO="DIVISAO">Europa</EM>\n\n      , acentuado pela "\n\n<OMITIDO>      <EM ID="H2-dftre765-17" CATEG="ACONTECIMENTO" TIPO="EFEMERIDE">Cativeiro Babilónica da igreja</EM></OMITIDO>\n\n      "\n\n      no papado de <EM ID="H2-dftre765-11" CATEG="ACONTECIMENTO" 

In [97]:
re.search('<EM .*>\s*\w+\s*</EM>', harem)

<re.Match object; span=(661, 742), match='<EM ID="H2-dftre765-37" CATEG="LOCAL" TIPO="HUMAN>

In [101]:
spacy.load('../data/external/spacy.zip')

OSError: [E053] Could not read meta.json from ../data/external/spacy.zip/meta.json

In [119]:
def train_spacy(data,iterations):
    TRAIN_DATA = data
    nlp = spacy.blank('pt')  # create blank Language class
    # create the built-in pipeline components and add them to the pipeline
    # nlp.create_pipe works for built-ins that are registered with spaCy
    if 'ner' not in nlp.pipe_names:
        ner = nlp.create_pipe('ner')
        nlp.add_pipe(ner, last=True)
       

    # add labels
    for _, annotations in TRAIN_DATA:
         for ent in annotations.get('entities'):
            ner.add_label(ent[2])

    # get names of other pipes to disable them during training
    other_pipes = [pipe for pipe in nlp.pipe_names if pipe != 'ner']
    with nlp.disable_pipes(*other_pipes):  # only train NER
        optimizer = nlp.begin_training()
        for itn in range(iterations):
            print("Statring iteration " + str(itn))
            random.shuffle(TRAIN_DATA)
            losses = {}
            for text, annotations in TRAIN_DATA:
                nlp.update(
                    [text],  # batch of texts
                    [annotations],  # batch of annotations
                    drop=0.2,  # dropout - make it harder to memorise data
                    sgd=optimizer,  # callable to update weights
                    losses=losses)
            print(losses)
    return nlp

In [139]:
train_spacy(h.dataset, 10)

Statring iteration 0


ValueError: [E103] Trying to set conflicting doc.ents: '(197, 204, 'TEMPO')' and '(178, 204, 'TEMPO')'. A token can only be part of one entity, so make sure the entities you're setting don't overlap.

In [1]:
slc = slice(10,20)